<a href="https://colab.research.google.com/github/dangdang2222/NLP_basic/blob/main/Torchtext_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torchtext

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import urllib.request
import pandas as pd

urllib.request.urlretrieve("https://raw.githubusercontent.com/LawrenceDuan/IMDb-Review-Analysis/master/IMDb_Reviews.csv", filename="IMDb_Reviews.csv")

('IMDb_Reviews.csv', <http.client.HTTPMessage at 0x7f5381ffb150>)

In [3]:
df = pd.read_csv('IMDb_Reviews.csv', encoding='latin1')
df.head()

,review,sentiment
0,My family and I normally do not watch local mo...,1
1,"Believe it or not, this was at one time the wo...",0
2,"After some internet surfing, I found the ""Home...",0
3,One of the most unheralded great works of anim...,1
4,"It was the Sixties, and anyone with long hair ...",0


In [4]:
len(df)

50000

In [5]:
train_df = df[:25000]
test_df = df[25000:]

In [6]:
train_df.to_csv("train_data.csv", index=False)
test_df.to_csv("test_data.csv", index=False)

In [8]:
from torchtext.legacy import data

In [14]:
TEXT = data.Field(sequential=True, #시퀀스
                  use_vocab=True, #단어 집합
                  tokenize=str.split, #어떤 토큰화 함수
                  lower=True, #소문자
                  batch_first=True, #미니 배치 차원을 맨 앞으로
                  fix_length=20) #최대 허용 길이(Padding)

LABEL = data.Field(sequential=False,
                   use_vocab=False,
                   batch_first=False,
                   is_target=True)

#Field는 전처리 진행에 대한 정의임
#torchtext.legacy.data.Field

In [12]:
from torchtext.legacy.data import TabularDataset

TabularDataset

데이터를 불러오면서 필드에서 정의했던 토큰화 방법으로** 토큰화**를 수행

- fields : data.Field에서 만든 필드 지정. (해당 필드를 호칭할 이름, 지정할 필드)

In [16]:
train_data, test_data = TabularDataset.splits(path='.',train='train_data.csv',test='test_data.csv', format='csv',
                                              fields=[('text',TEXT),('label',LABEL)],skip_header=True)

In [19]:
print(train_data, len(train_data))

<torchtext.legacy.data.dataset.TabularDataset object at 0x7f5318763550> 25000


In [20]:
print(test_data, len(test_data))

<torchtext.legacy.data.dataset.TabularDataset object at 0x7f53187daf10> 25000


In [21]:
train_data[0]

In [22]:
vars(train_data[0])

{'text': ['my',
  'family',
  'and',
  'i',
  'normally',
  'do',
  'not',
  'watch',
  'local',
  'movies',
  'for',
  'the',
  'simple',
  'reason',
  'that',
  'they',
  'are',
  'poorly',
  'made,',
  'they',
  'lack',
  'the',
  'depth,',
  'and',
  'just',
  'not',
  'worth',
  'our',
  'time.<br',
  '/><br',
  '/>the',
  'trailer',
  'of',
  '"nasaan',
  'ka',
  'man"',
  'caught',
  'my',
  'attention,',
  'my',
  'daughter',
  'in',
  "law's",
  'and',
  "daughter's",
  'so',
  'we',
  'took',
  'time',
  'out',
  'to',
  'watch',
  'it',
  'this',
  'afternoon.',
  'the',
  'movie',
  'exceeded',
  'our',
  'expectations.',
  'the',
  'cinematography',
  'was',
  'very',
  'good,',
  'the',
  'story',
  'beautiful',
  'and',
  'the',
  'acting',
  'awesome.',
  'jericho',
  'rosales',
  'was',
  'really',
  'very',
  'good,',
  "so's",
  'claudine',
  'barretto.',
  'the',
  'fact',
  'that',
  'i',
  'despised',
  'diether',
  'ocampo',
  'proves',
  'he',
  'was',
  'effect

vars() : 인덱스와 샘플 확인

In [23]:
train_data.fields.items()

dict_items([('text', <torchtext.legacy.data.field.Field object at 0x7f532fa91250>), ('label', <torchtext.legacy.data.field.Field object at 0x7f532f5112d0>)])

토큰화 -> **단어 집합** -> 정수 인코딩

build_vocab
- min_freq : 단어의 최소 등장 빈도
- max_size : 단어 집합의 최대 크기(이때 자동으로 <unk>:0 <pad>:1 추가됌

In [25]:
TEXT.build_vocab(train_data, min_freq=10, max_size=10000)

In [26]:
len(TEXT.vocab)

10002

In [28]:
TEXT.vocab.stoi

defaultdict(<bound method Vocab._default_unk_index of <torchtext.vocab.Vocab object at 0x7f53310c3a90>>,
            {'<unk>': 0,
             '<pad>': 1,
             'the': 2,
             'a': 3,
             'and': 4,
             'of': 5,
             'to': 6,
             'is': 7,
             'in': 8,
             'i': 9,
             'this': 10,
             'it': 11,
             'that': 12,
             '/><br': 13,
             'was': 14,
             'as': 15,
             'for': 16,
             'with': 17,
             'but': 18,
             'on': 19,
             'movie': 20,
             'his': 21,
             'not': 22,
             'are': 23,
             'you': 24,
             'film': 25,
             'have': 26,
             'he': 27,
             'be': 28,
             'at': 29,
             'one': 30,
             'by': 31,
             'an': 32,
             'they': 33,
             'from': 34,
             'who': 35,
             'all': 36,
             'like

DataLoader : 
데이터셋에서 미니 배치만큼 데이터를 로드

In [30]:
from torchtext.legacy.data import Iterator

In [34]:
batch_size=5
train_loader = Iterator(dataset=train_data, batch_size=batch_size)
test_loader = Iterator(dataset=test_data, batch_size=batch_size)

In [35]:
len(train_loader)

5000

첫번째 미니 배치를 받아와서 batch에 저장

In [38]:
batch = next(iter(train_loader))

In [39]:
batch


[torchtext.legacy.data.batch.Batch of size 5]
	[.text]:[torch.LongTensor of size 5x20]
	[.label]:[torch.LongTensor of size 5]

In [40]:
batch.text

tensor([[  80,   68,   11,    0,    2,  121,   34,    2,   82,   25,    4,  446,
          116,   51,    0,    0,    0,    2,    0,    0],
        [  30,    5,    2,   51,  242,  127,  186,  699, 2527,    0,    0,   13,
            0, 2527,    0,    7,    8, 3775,    5,    2],
        [3403, 2193,    0,    8,    0,    0,   26,    3,    0,    0,    0,   17,
            2,    0,    5,    0,   18,   48,  597,   50],
        [2301, 4257,  941,    8,   65,    0,    0,  179,    0,    0, 2552,   72,
            2, 3374,    8, 1526,    5,    0, 1108,    0],
        [  55,   82,  927,   29, 6750,   94,    3,  156,  182,    5,  712,   10,
         3090,    0,   13,    0,  202,   10,   19,    2]])

batch size가 5이고 Field에서 fix_length가 20이므로 5 * 20

batch_size * fix_length

---

형태소 분석기 Mecab

In [41]:
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 115, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 115 (delta 11), reused 10 (delta 3), pack-reused 91
Receiving objects: 100% (115/115), 1.27 MiB | 11.02 MiB/s, done.
Resolving deltas: 100% (50/50), done.
/content/Mecab-ko-for-Google-Colab
Installing konlpy.....
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 1.3 MB/s 
     |████████████████████████████████| 453 kB 58.9 MB/s 
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2022-09-07 05:21:35--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 18.205.93.2, 18.205.93.1, 18.205.93.0, ...
Connecting to bitbucket.org (bitbucket.org)|18.205.93.2|:443

In [42]:
import urllib.request
import pandas as pd

urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt")
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", filename="ratings_test.txt")

('ratings_test.txt', <http.client.HTTPMessage at 0x7f52ebcc16d0>)

In [43]:
train_df = pd.read_table('ratings_train.txt')
test_df = pd.read_table('ratings_test.txt')

In [44]:
train_df.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [45]:
from torchtext.legacy import data # torchtext.data 임포트
from konlpy.tag import Mecab

tokenizer = Mecab()

In [47]:
ID = data.Field(sequential=False,
                use_vocab=False)

TEXT = data.Field(sequential=True,
                  use_vocab=True,
                  tokenize = tokenizer.morphs,
                  lower=True,
                  batch_first = True,
                  fix_length = 20)

LABEL = data.Field(sequential =False,
                   use_vocab = False,
                   is_target=True)

In [49]:
from torchtext.legacy.data import TabularDataset
train_data, test_data = TabularDataset.splits(path='.',train='ratings_train.txt',test='ratings_test.txt',format='tsv',
                                              fields=[('id',ID),('text',TEXT),('label',LABEL)],skip_header=True)

In [50]:
print(len(train_data),len(test_data))

150000 50000


In [51]:
train_data[0]

In [52]:
vars(train_data[0])

{'id': '9976970',
 'text': ['아', '더', '빙', '.', '.', '진짜', '짜증', '나', '네요', '목소리'],
 'label': '0'}

In [53]:
TEXT.build_vocab(train_data, min_freq=10, max_size=10000)

In [54]:
len(TEXT.vocab)

10002

In [55]:
TEXT.vocab.stoi

defaultdict(<bound method Vocab._default_unk_index of <torchtext.vocab.Vocab object at 0x7f52e8b17c10>>,
            {'<unk>': 0,
             '<pad>': 1,
             '.': 2,
             '이': 3,
             '는': 4,
             '영화': 5,
             '다': 6,
             '고': 7,
             '하': 8,
             '도': 9,
             '의': 10,
             '가': 11,
             '은': 12,
             '에': 13,
             '을': 14,
             '보': 15,
             '한': 16,
             '..': 17,
             '게': 18,
             ',': 19,
             '들': 20,
             '!': 21,
             '지': 22,
             '를': 23,
             '있': 24,
             '없': 25,
             '?': 26,
             '좋': 27,
             '나': 28,
             '었': 29,
             '만': 30,
             '는데': 31,
             '너무': 32,
             '봤': 33,
             '적': 34,
             '안': 35,
             '정말': 36,
             '로': 37,
             '음': 38,
             '으로': 39,
           

In [56]:
from torchtext.legacy.data import Iterator

In [57]:
batch_size=5
train_loader = Iterator(dataset=train_data, batch_size=batch_size)
test_loader = Iterator(dataset=test_data, batch_size=batch_size)

In [58]:
len(train_loader)

30000

In [59]:
batch = next(iter(train_loader))

In [60]:
batch.text

tensor([[1677,  112,  179,    5,    2,    2, 1190,   12, 2651,  408, 3902,    6,
            1,    1,    1,    1,    1,    1,    1,    1],
        [7819, 6618,   13, 1148,   18,  599,   14,  133,   56,   24,   77,   71,
         2306,   15,    4,   40,   14,  826,  227,   10],
        [ 415,   70,  410,  130,   50,   96,  599, 1624,   23,   74,   56,   11,
           24,  309,   26,  382,   89,   20,   10,    0],
        [ 198,   33,    6,    2,   17,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1],
        [1055,   15,   51,  404,   49,   31,  139,   96,  148,  303,   21,  541,
            0,  237,   21,    1,    1,    1,    1,    1]])